In [1]:
import numpy as np
import os
import cv2
import pandas as pd

from sklearn.cross_validation import train_test_split
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam, Adadelta
from keras.callbacks import ModelCheckpoint, EarlyStopping

from keras import backend as K
K.set_image_dim_ordering('th')


C:\Users\skumarravindran\AppData\Local\Continuum\Anaconda2\envs\py35gpu1\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [2]:
image_size = 448 # size to work best with the new NN 

def load_train():
    X_train = []
    y_train = []
    dt = pd.read_csv('train_data.csv')
    print('Read train images')
    for index, row in dt.iterrows():
        image_path = os.path.join('trainImages', str(row['name']))
        print(image_path)
        img = cv2.resize(cv2.imread(image_path, cv2.IMREAD_COLOR), (image_size, image_size) ).astype(np.float32)
        img = img.transpose((2,0,1))
        X_train.append(img)
        y_train.append( [ row['likes_count'] ] )
    return X_train, y_train

In [3]:
def read_and_normalize_train_data():
    train_data, train_target = load_train()
    train_data = np.array(train_data, dtype=np.float32)
    train_target = np.array(train_target, dtype=np.float32)
    m = train_data.mean()
    s = train_data.std()

    print ('Train mean, sd:', m, s )
    # normalization of the image
    train_data -= m
    train_data /= s
    print('Train shape:', train_data.shape)
    print(train_data.shape[0], 'train samples')
    return train_data, train_target

In [8]:
# version 1: Model 1
def create_model():
    nb_filters = 16
    nb_conv = 5

    model = Sequential()
    model.add(Convolution2D(nb_filters, nb_conv, nb_conv,
                            border_mode='valid',
                            input_shape=(3, image_size, image_size) ) )
    model.add(Activation('relu'))

    model.add(Convolution2D(nb_filters, nb_conv, nb_conv))
    model.add(Activation('relu'))

    model.add(Convolution2D(nb_filters, nb_conv, nb_conv))
    model.add(Activation('relu'))

    model.add(Convolution2D(nb_filters, nb_conv, nb_conv))
    model.add(Activation('relu'))
    model.add(Dropout(0.25))

    model.add(Convolution2D(nb_filters*2, nb_conv, nb_conv))
    model.add(Activation('relu'))

    model.add(Convolution2D(nb_filters*2, nb_conv, nb_conv))
    model.add(Activation('relu'))

    model.add(Convolution2D(nb_filters*2, nb_conv, nb_conv))
    model.add(Activation('relu'))
    
    model.add(Convolution2D(nb_filters*2, nb_conv, nb_conv))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Dense(1))
    model.add(Activation('linear'))

    return model

In [4]:
# version 2 : Model 2
def create_model():
    model = Sequential()
    model.add(Convolution2D(32, 7, 7, border_mode='same', subsample=(2, 2), init='glorot_uniform', input_shape=(3, image_size, image_size))) # 224
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2))) # 112

    model.add(Convolution2D(64, 3, 3, border_mode='same', subsample=(1, 1), init='glorot_uniform', )) # 56
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2))) # 28

    model.add(Convolution2D(128, 3, 3, border_mode='same', subsample=(1, 1), init='glorot_uniform', ))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2))) # 28

    model.add(Convolution2D(256, 3, 3, border_mode='same', subsample=(1, 1), init='glorot_uniform', ))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2))) # 28
    
    model.add(Convolution2D(512, 3, 3, border_mode='same', subsample=(1, 1), init='glorot_uniform', ))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(7, 7))) # 14

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    

    model.add(Dense(1))
    model.add(Activation('linear'))

    return model

In [25]:
# version 3 : Model 3
def create_model():
    nb_filters = 16
    nb_conv = 5

    model = Sequential()
    model.add(Convolution2D(nb_filters, nb_conv, nb_conv,
                            border_mode='valid', subsample=(2, 2),
                            input_shape=(3, image_size, image_size) ) )
    model.add(Activation('relu'))
    
    model.add(Convolution2D(nb_filters, nb_conv, nb_conv))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Convolution2D(nb_filters, nb_conv, nb_conv))
    model.add(Activation('relu'))
    
    model.add(Convolution2D(nb_filters, nb_conv, nb_conv))
    model.add(Activation('relu'))
    model.add(Dropout(0.25))

    model.add(Convolution2D(nb_filters*2, nb_conv, nb_conv))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Convolution2D(nb_filters*2, nb_conv, nb_conv))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Convolution2D(nb_filters*2, nb_conv, nb_conv))
    model.add(Activation('relu'))
    
    model.add(Convolution2D(nb_filters*2, nb_conv, nb_conv))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Dense(1))
    model.add(Activation('linear'))

    return model

In [ ]:
# batch normalization
# command to add
#The general use case is to use BN between the linear and non-linear layers in your network
#FYI apparently batch normalization works better in practice after the activation function
#from keras.layers.normalization import BatchNormalization
#model.add(BatchNormalization())

In [5]:
def train_model(batch_size = 50, nb_epoch = 500):
    num_samples = 1000
    cv_size = 100

    train_data, train_target = read_and_normalize_train_data()
    train_data = train_data[0:num_samples,:,:,:]
    train_target = train_target[0:num_samples]

    X_train, X_valid, y_train, y_valid = train_test_split(train_data, train_target, test_size=cv_size, random_state=56742)

    model = create_model()
    ####### Saving the best Model for re-use on fitting
    #version: 1 and 2
    model.compile(loss='mean_squared_error', optimizer=Adadelta())
    #checkpointer = ModelCheckpoint(filepath="C:/Users/skumarravindran/Documents/keras_save_model/weights.hdf5", verbose=1, save_best_only=True)
    checkpointer = [
                    # Stop if the accuracy is not improving after 7 iterations
                    EarlyStopping(monitor='val_loss', patience=7, verbose=0),
                    # Saving the best model and re-use it while prediction
                    ModelCheckpoint(filepath="C:/Users/skumarravindran/Documents/keras_save_model/weights.hdf5", verbose=1, save_best_only=True),
                ]
    #######
    model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=nb_epoch, verbose=1, validation_data=(X_valid, y_valid), callbacks=checkpointer )
    
    #####Load weights from saved model
    model.load_weights('C:/Users/skumarravindran/Documents/keras_save_model/weights.hdf5')
    ####
    
    predictions_valid = model.predict(X_valid, batch_size=50, verbose=1)
    compare = pd.DataFrame(data={'original':y_valid.reshape((cv_size,)),
             'prediction':predictions_valid.reshape((cv_size,))})
    compare.to_csv('compare.csv')

    return model

train_model(nb_epoch = 500)

Read train images
trainImages\19955218_1079660242164715_8942226110473568256_n.jpg
trainImages\19984245_294558477675146_1938364997482577920_n.jpg
trainImages\19933101_1908348909404541_6909127414480109568_n.jpg
trainImages\19932661_1971632416383297_8971921282429878272_n.jpg
trainImages\19984849_1504065326309503_3355268536564973568_n.jpg
trainImages\19932977_318600325231429_7765716787211010048_n.jpg
trainImages\19761936_447469295621631_6731500598048325632_n.jpg
trainImages\19931939_356577494761007_2849061577934503936_n.jpg
trainImages\19932560_881846375287698_8808400087785930752_n.jpg
trainImages\19762061_453187828382463_4926556637902143488_n.jpg
trainImages\19764384_1367861406666469_3096634835884572672_n.jpg
trainImages\19764445_451710105196462_505015337134063616_n.jpg
trainImages\19761852_142594382983584_9145026520501190656_n.jpg
trainImages\19624650_623899511135465_2497541710465204224_n.jpg
trainImages\19622958_429038887495771_5574578052232380416_n.jpg
trainImages\19534779_113140039308

C:\Users\skumarravindran\AppData\Local\Continuum\Anaconda2\envs\py35gpu1\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (7, 7), strides=(2, 2), padding="same", input_shape=(3, 448, 4..., kernel_initializer="glorot_uniform")`
  after removing the cwd from sys.path.
C:\Users\skumarravindran\AppData\Local\Continuum\Anaconda2\envs\py35gpu1\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), strides=(1, 1), padding="same", kernel_initializer="glorot_uniform")`
  
C:\Users\skumarravindran\AppData\Local\Continuum\Anaconda2\envs\py35gpu1\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), strides=(1, 1), padding="same", kernel_initializer="glorot_uniform")`
  if sys.path[0] == '':
C:\Users\skumarravindran\AppData\Local\Continuum\Anaconda2\envs\py35gpu1\lib\site-packages\ipykernel_launcher

Train on 900 samples, validate on 100 samples
Epoch 1/500
900/900 [==============================] - 31s - loss: 436667508508.4445 - val_loss: 448826359808.0000
Epoch 2/500
900/900 [==============================] - 24s - loss: 471300973454.2222 - val_loss: 452320378880.0000
Epoch 3/500
900/900 [==============================] - 24s - loss: 205555681052.4444 - val_loss: 202680492032.0000
Epoch 4/500
900/900 [==============================] - 24s - loss: 157447193486.2222 - val_loss: 155742363648.0000
Epoch 5/500
900/900 [==============================] - 24s - loss: 156788217628.4444 - val_loss: 174103478272.0000
Epoch 6/500
900/900 [==============================] - 24s - loss: 144919896519.1111 - val_loss: 155664662528.0000
Epoch 7/500
900/900 [==============================] - 24s - loss: 131803713536.0000 - val_loss: 175908003840.0000
Epoch 8/500
900/900 [==============================] - 24s - loss: 135885893176.8889 - val_loss: 180906295296.0000
Epoch 9/500
900/900 [=============